In [0]:
import pandas as pd
import numpy as np
import sys
import re
import gc
import csv

!pip install pycld2

from google.colab import drive
drive.mount('/content/drive')

!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
  
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

import pycld2 as cld2
languages = list(set([w[0].lower() for w in cld2.LANGUAGES]))
languages

In [0]:
file_list = drive.ListFile({'q': "'1IMT5wkteYw7oVhN5exjoHmFtGaWSVu28' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: IRAhandle_tweets_8.csv, id: 1tncgM0Qk8tVwnlDpcS6ExcKLgXSQFivr
title: IRAhandle_tweets_6.csv, id: 1Zj7y_-3A-7Fos-zGCHEKgPBoQvjmysCS
title: IRAhandle_tweets_7.csv, id: 1jI5nQyznspfgk6x1RqJeYs184R8rDAH3
title: IRAhandle_tweets_9.csv, id: 14fs5-wJUPRJ5_n_KNWOFkXOF_JgQBcvn
title: IRAhandle_tweets_5.csv, id: 1zqwycOdInUlWqs6n1cg1O9ahs-LdfDH-
title: IRAhandle_tweets_3.csv, id: 1Rk7XNjnBzeCPi2o-TV1pD8qbMFOxrnXH
title: IRAhandle_tweets_4.csv, id: 1QSArmXCXdkfWHlcTVqPwnzdAjjkOenF7
title: IRAhandle_tweets_2.csv, id: 1bNSGGniOObCfzTZ085FyFVjqNpC5cRnD
title: IRAhandle_tweets_1.csv, id: 1TEkomSxdkUuhkpWTvB1YNZFsvEVP_Pwk


In [0]:
trolldf = pd.DataFrame()

for file in file_list :
  
  downloaded = drive.CreateFile({'id' : file['id']})
  
  downloaded.GetContentFile(file['title'])
  
  trolldf = trolldf.append( pd.read_csv(file['title'], usecols = ['author','language']), ignore_index = True, sort = False)
           
trolldf.head()

,author,language
0,STAYWOKE88,English
1,STAYWOKE88,English
2,STAYWOKE88,English
3,STAYWOKE88,English
4,STAYWOKE88,English


In [0]:
trolldf['language'] = trolldf['language'].apply(str.lower)
trolldf.head()

,author,language
0,STAYWOKE88,english
1,STAYWOKE88,english
2,STAYWOKE88,english
3,STAYWOKE88,english
4,STAYWOKE88,english


In [0]:
handle1 = open('/tmp/train.csv', 'w')
handle2 = open('/tmp/test.csv', 'w')

writer1 = csv.writer(handle1)
writer2 = csv.writer(handle2)

temp = ['author']

for s in languages :
    temp.append(s)
    
temp.append('troll')

writer1.writerow(temp)
writer2.writerow(temp)

! cat /tmp/train.csv
! cat /tmp/test.csv

In [0]:
troll_zeros = [0] * len(languages)

troll_lang_dict = dict(zip(languages, troll_zeros))

troll_per_profile_lang = trolldf.groupby('author')['language']

troll_per_profile_lang.head()

In [0]:
troll_x = []
troll_idx = 0


for troll_key, grp in troll_per_profile_lang:
    troll_idx = troll_idx + 1
    print('id : ', troll_idx, ' author :  ' , troll_key)
    
    troll_x.append(troll_key)
    
    length = len(grp)
    
    for s in grp:
        if s in troll_lang_dict:
            troll_lang_dict[s] = troll_lang_dict[s] + 1
            
    for val in list(troll_lang_dict.values()) :
        troll_x.append(val/length)
            
    troll_x.append(1)
    
    if troll_idx < 2130 :
        writer1.writerow(troll_x)
    else :
        writer2.writerow(troll_x)
    
    troll_x.clear()


    
    for troll_k in troll_lang_dict.keys():
        
        troll_lang_dict[troll_k] = 0
    

TROLL - NORMAL - BORDER

In [0]:
file_list = drive.ListFile({'q': "'1sFcNq_jqE8-vdmC1TlXrh2plS8qy84PZ' in parents and trashed=false"}).GetList()

for file1 in file_list:
  
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: nov29.csv, id: 12NSIxIbgiQd8N3i49lcK4DeBpaV6hCWc
title: nov28.csv, id: 1huayYTurbpgpsVI99IoKVyUBkdKGhz_N
title: nov27.csv, id: 1AAAUuIdyq1TjZgO54Njl3s4E4hDzeTAK
title: nov30.csv, id: 1cmPemTfEF61B2TWpwO1f8h9WQxIdxKf8


In [0]:
normaldf = pd.DataFrame()

for file in file_list :
  
  downloaded = drive.CreateFile({'id' : file['id']})
  
  downloaded.GetContentFile(file['title'])
  
  normaldf = normaldf.append( pd.read_csv(file['title'], usecols = ['author_id','language_code']), ignore_index = True, sort = False)
           
normaldf.head()

,author_id,language_code
0,225551004,en
1,184621766,en
2,743166519157227520,en
3,53706363,th
4,2418453842,ja


In [0]:
reversed_dict = dict([(v.lower(),k.lower()) for (k, v) in list(set(cld2.LANGUAGES))])
def code_to_lang(code):
    return reversed_dict[code]

In [0]:
normaldf.columns = ['author', 'language']
normaldf['language'] = normaldf['language'].apply(code_to_lang)
normaldf.head()

,author,language
0,225551004,english
1,184621766,english
2,743166519157227520,english
3,53706363,thai
4,2418453842,japanese


In [0]:
normal_zeros = [0] * len(languages)

normal_lang_dict = dict(zip(languages, normal_zeros))

normal_per_profile_lang = normaldf.groupby('author')['language']

In [0]:
normal_x = []
normal_idx = 0

for normal_key, grp in normal_per_profile_lang:
    normal_idx = normal_idx + 1
    
    normal_x.append(normal_key)
    
    
    length = len(grp)
    
    for s in grp:
        
        if s in normal_lang_dict:
            normal_lang_dict[s] = normal_lang_dict[s] + 1
            
    for val in list(normal_lang_dict.values()):
        normal_x.append(val / length)
            
    normal_x.append(0)
    
    if normal_idx < 1500000 :
        if normal_idx < 1125000 :
            writer1.writerow(normal_x)
        else :
            writer2.writerow(normal_x)
    else :
        break

    normal_x.clear()


    
    for normal_k in normal_lang_dict.keys():
        
        normal_lang_dict[normal_k] = 0
    

MODEL - TRAINING - TESTING

In [0]:
languages.insert(0, 'author')

X_train = pd.read_csv('/tmp/train.csv', usecols = languages)
X_test  = pd.read_csv('/tmp/test.csv',  usecols = languages)

y_train = pd.read_csv('/tmp/train.csv', usecols = ['author', 'troll'])
y_test  = pd.read_csv('/tmp/test.csv',  usecols = ['author', 'troll'])

X_train.set_index('author', drop=True, append=False, inplace=True, verify_integrity=False)
X_test.set_index ('author', drop=True, append=False, inplace=True, verify_integrity=False)

y_train.set_index('author', drop=True, append=False, inplace=True, verify_integrity=False)
y_test.set_index ('author', drop=True, append=False, inplace=True, verify_integrity=False)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:

Log_clf = LogisticRegression(solver = 'lbfgs').fit(X_train, y_train)

predicted = Log_clf.predict(X_test)
print('Accuracy : {:.2f}', format(accuracy_score(y_test, predicted)))
print('Precision : {:.2f}', format(precision_score(y_test, predicted)))
print('Recall : {:.2f}', format(recall_score(y_test, predicted)))
print('f1-score : {:.2f}', format(f1_score(y_test, predicted)))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy : {:.2f} 0.0
Precision : {:.2f} 0.0
Recall : {:.2f} 0.0
f1-score : {:.2f} 0.0


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:

Dt_clf = DecisionTreeClassifier().fit(X_train, y_train)

In [0]:
predicted = Dt_clf.predict(X_test)
print('Accuracy : {:.2f}', format(accuracy_score(y_test, predicted)))
print('Precision : {:.2f}', format(precision_score(y_test, predicted)))
print('Recall : {:.2f}', format(recall_score(y_test, predicted)))
print('f1-score : {:.2f}', format(f1_score(y_test, predicted)))

Accuracy : {:.2f} 1.0
Precision : {:.2f} 1.0
Recall : {:.2f} 1.0
f1-score : {:.2f} 1.0


In [0]:

GBDT_clf = GradientBoostingClassifier().fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
predicted = GBDT_clf.predict(X_test)
print('Accuracy : {:.2f}', format(accuracy_score(y_test, predicted)))
print('Precision : {:.2f}', format(precision_score(y_test, predicted)))
print('Recall : {:.2f}', format(recall_score(y_test, predicted)))
print('f1-score : {:.2f}', format(f1_score(y_test, predicted)))

Accuracy : {:.2f} 0.9980863357988283
Precision : {:.2f} 0.0
Recall : {:.2f} 0.0
f1-score : {:.2f} 0.0


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
